# First Deliverable - Extract Cities and Countries

import of used modules

In [11]:
import geonamescache
import re
import unidecode
import pandas as pd

City and country data is held within dictionarys and can be summoned thus

In [2]:
gc = geonamescache.GeonamesCache()
cities = gc.get_cities()
countries = gc.get_countries()

The data held within has nested dictionarys to the best way to extract specific fields is with a for each loop.

The dictionary called with get_cities() for e.g has the form " { key: id number, Value: { "id No.", "city Name"... } } etc. and each city name will need to be placed in one long regex pattern for matching e.g

Declaring the start string for the regex pattern is nesessary before then entering the loop

In [3]:
cityRegex = r"\b("
for city in cities:
    cityClense = re.sub(r"\(|\)", "", cities[city]['name'])
    cityRegex = cityRegex + unidecode.unidecode(cityClense) + "|"
    
cityRegex = cityRegex[:-1] + r")\b"

Note that the city names will have unicode characters with accents and graves which will need to be decoded with the unidecode library.  Some of the city names contain parenthesis which will affect the match output later on and have been replaced with re.sub().

To make the matching process much quicker, the regex patter is compiled...

In [4]:
cityRegexCompiled = re.compile(cityRegex, flags=re.IGNORECASE)

The same process needs to be carried out for the country dictionary

In [5]:
countryRegex = r"\b("
for country in countries:
    countryClense = re.sub(r"\(|\)", "", countries[country]['name'])
    countryRegex = countryRegex + unidecode.unidecode(countryClense) + "|"

countryRegex = countryRegex[:-1] + r")\b"
countryRegexCompiled = re.compile(countryRegex, flags=re.IGNORECASE)


Now we have regex pattern, we need to match this against all the headlines.  A record of matches or non matches is kept in a new dictionary declared before the matching process

In [6]:
dictionary = { "headline": [], "city": [], "country": [] }

I have used the regex method 'findall()' as this will take into account multiple name matches per headline.  This returns a list of matched results and an empty list if no matches.

The process below takes each line and pattern matches against both city and country.

If the match is 1 then the city name is added to our dictionary.

If the pattern matches 2 names in the same headline the longest string found is returned so that partial matches like 'San' or 'Of' don't get taken into account. 

In [7]:
with open(r'data/headlines.txt', 'r') as data:
    for line in data:
        dictionary['headline'].append(line.rstrip('\n'))
        countryMatch = re.findall(countryRegexCompiled, line)
        if len(countryMatch) == 1:
            dictionary['country'].append(countryMatch[0])
        elif len(countryMatch) > 1:
            dictionary['country'].append(max(countryMatch, key=len))
        else:
            dictionary['country'].append("NaN")

        cityMatch  = re.findall(cityRegexCompiled, line)
        if len(cityMatch) == 1:
            dictionary['city'].append(cityMatch[0])
        elif len(cityMatch) > 1:
            dictionary['city'].append(max(cityMatch, key=len))
        else:
            dictionary['city'].append("NaN")


Our dictionary, now filled with matches or non matches can be converted to a pandas dataframe...

In [8]:
df = pd.DataFrame(dictionary)

In [9]:
pd.set_option('max_rows', None)

In [12]:
df.style

,headline,city,country
0,Zika Outbreak Hits Miami,Miami,NaN
1,Could Zika Reach New York City?,New York City,NaN
2,First Case of Zika in Miami Beach,Miami,NaN
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,NaN
5,Trinidad confirms first Zika case,Trinidad,NaN
6,Zika Concerns are Spreading in Houston,Houston,NaN
7,Geneve Scientists Battle to Find Cure,Geneve,NaN
8,The CDC in Atlanta is Growing Worried,Atlanta,NaN
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,NaN
